In [1]:
import polars as pl

In [18]:
data1 = pl.read_excel(source='data/data1.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data2 = pl.read_excel(source='data/data2.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data3 = pl.read_excel(source='data/data3.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data4 = pl.read_excel(source='data/data4.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")
data5 = pl.read_excel(source='data/data5.xlsx', engine="xlsx2csv", sheet_name="Результаты поиска")

Оставляем только те строки, где известен победитель и есть ИНН заказчика и поставщика.

In [225]:
fdata1 = data1.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata2 = data2.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata3 = data3.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata4 = data4.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())
fdata5 = data5.filter(pl.col("ИНН поставщика").is_not_null() & pl.col("ИНН заказчика").is_not_null())

In [337]:
data = pl.concat([fdata1, fdata2, fdata3, fdata5])
data = data.with_columns(pl.col("ИНН поставщика").str.to_integer())
data = pl.concat([data, fdata4])
data.shape

(168453, 18)

Убираем дупликаты

In [ ]:
data = data.unique(maintain_order=True)
data.shape

In [338]:
data = data.rename({'Дата \nокончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ':'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                    'Дата начала подачи заявок/Дата начала исполнения контракта / Дата публикации ППГ' : 'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                    'Дата  окончания проведения торгов' : 'Дата окончания проведения торгов'})

Оставляем только те строки, где известно снижение на торгах

In [339]:
data = data.filter(pl.col("Снижение на торгах,%").is_not_null())
data.shape

(135397, 18)

Уберем нереалистичные значения в ставках в Снижение на торгах,% 

In [340]:
data = data.filter((pl.col('Снижение на торгах,%')>=-100))
data.shape

(135032, 18)

Теперь работаем с датами

In [341]:
ddata = data.with_columns(pl.col('Дата публикации').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').str.to_date(format="%m-%d-%y", strict=False))
ddata = ddata.with_columns(pl.col('Дата окончания проведения торгов').str.to_date(format="%m-%d-%y", strict=False))

In [351]:
data = ddata.filter(pl.col('Дата публикации').is_not_null() & 
                         pl.col('Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ').is_not_null() & 
                         pl.col('Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ').is_not_null() & 
                         pl.col('Дата окончания проведения торгов').is_not_null())
data.shape

(123013, 18)

Отбираем тендеры, где известен победитель

In [354]:
data = data.with_columns(pl.when(pl.col('Победитель')==pl.lit('Победитель')).then(1).otherwise(0).alias('Есть победитель'))
data.shape

(123013, 19)

In [358]:
data_group = data.group_by(['Уровень',
                            'Заказчик',
                            'ИНН заказчика',
                            'Стоимость \n(руб.) Заказчик',
                            'Сфера деятельности',
                            'Регион поставки',
                            'Город поставки',
                            'Дата публикации',
                            'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                            'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                            'Дата окончания проведения торгов',
                            'Форма публикации',
                            'Тип торгов']).agg(pl.col('Есть победитель').sum().alias('Num winner'))
data_group.shape

(49825, 14)

In [348]:
dataw = data.filter(pl.col('Победитель')==pl.lit('Победитель')).select(['Уровень',
                                                                        'Заказчик',
                                                                        'ИНН заказчика',
                                                                        'Стоимость \n(руб.) Заказчик',
                                                                        'Сфера деятельности',
                                                                        'Регион поставки',
                                                                        'Город поставки',
                                                                        'Дата публикации',
                                                                        'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                                                                        'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                                                                        'Дата окончания проведения торгов',
                                                                        'Форма публикации',
                                                                        'Тип торгов'])
dataw.shape

(12561, 13)

In [359]:
datanew = data.join(data_group, 
                    how='left',
                    on=['Уровень',
                        'Заказчик',
                        'ИНН заказчика',
                        'Стоимость \n(руб.) Заказчик',
                        'Сфера деятельности',
                        'Регион поставки',
                        'Город поставки',
                        'Дата публикации',
                        'Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ',
                        'Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ',
                        'Дата окончания проведения торгов',
                        'Форма публикации',
                        'Тип торгов'])

datanew.shape

(123013, 20)

In [357]:
df = pl.DataFrame(
    {
        "foo": [1, 2, 3, 4],
        "bar": [6.0, 7.0, 8.0, 9.0],
        "ham": ["a", "b", "c", "a"],
    }
)
other_df = pl.DataFrame(
    {
        "apple": ["x", "y", "z"],
        "ham": ["a", "b", "d"],
    }
)
df.join(other_df, on="ham")

foo,bar,ham,apple
i64,f64,str,str
1,6.0,"""a""","""x"""
2,7.0,"""b""","""y"""
4,9.0,"""a""","""x"""


In [362]:
data = datanew.filter(pl.col('Num winner').is_not_null())
data.shape

(32449, 20)

In [363]:
data = data.drop(["Есть победитель", "Num winner"])
data.head(2)

Уровень,Заказчик,ИНН заказчика,Стоимость (руб.) Заказчик,Сфера деятельности,Регион поставки,Город поставки,Дата публикации,Дата окончания приема заявок / Дата планового окончания исполнения контракта / Плановая дата публикации лота по ППГ,Дата начала подачи заявок / Дата начала исполнения контракта / Дата публикации ППГ,Дата окончания проведения торгов,Поставщик,ИНН поставщика,Победитель,Статус допуска,"Снижение на торгах,%",Форма публикации,Тип торгов
i64,str,i64,f64,str,str,str,date,date,date,date,str,i64,str,str,f64,str,str
1,"""АО ""РОССЕТИ ЯНТАРЬ""""",3903007130,2.9783e7,"""[ОКПД2 43.21] Работы электромо…","""Калининградская область""","""Калининград""",2023-05-24,2023-05-31,2023-05-24,2023-06-26,"""ООО ""ОРИОН""""",3906177059,null,"""Допущен""",0.00006,"""Торговая процедура""","""Запрос цен"""
1,"""МУП ""РЕСУРС""""",6614001840,151972.23,"""[ОКПД2 28.14] Арматура трубопр…","""Свердловская область""","""Свердловская область""",2023-05-30,2023-05-31,2023-05-30,2023-05-30,"""ООО ""САНТЕХКОМПЛЕКТ-ТАГИЛ""""",6623019699,"""Победитель""","""Допущен""",0.0,"""Торговая процедура""","""Закупка у единственного постав…"


Добавляем реестр недобросовестных поставщиков

In [365]:
notrust = pl.read_json('notrust_list/res.json')

In [366]:
notrust = notrust.rename({'suplier_name':'Поставщик', 'suplier_inn':'ИНН поставщика', 'no_trust_now':'РНП сейчас', 'no_trust_before':'РНП ранее'})
notrust = notrust.with_columns(pl.col('ИНН поставщика').str.to_integer())

In [367]:
data = data.join(other=notrust.select(['ИНН поставщика', 'РНП сейчас', 'РНП ранее']), on='ИНН поставщика', how='left')
data = data.with_columns(pl.col('РНП сейчас').fill_null(0), pl.col('РНП ранее').fill_null(0))

In [370]:
data.filter(pl.col('РНП ранее')==1).shape

(15, 20)

Записываем результат

In [371]:
data.write_excel( dtype_formats={pl.Date: "yyyy-mm-dd"})

In [372]:
data.shape

(32455, 20)